In [4]:
! pip install pyspark


In [6]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

In [8]:
from pyspark import SparkConf
conf = SparkConf().setAppName("TP3_SparkSQL").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)

/usr/local/lib/python3.12/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [9]:
fichier = sc.textFile("/content/ngram.csv")


In [43]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Séparer les colonnes
tableau = fichier.map(lambda ligne: ligne.split("\t"))


In [44]:
tableau.take(5)


[['! $17.95', '1985', '1', '1', '1'],
 ['! $17.95', '1987', '1', '1', '1'],
 ['! $17.95', '1990', '1', '1', '1'],
 ['! $17.95', '1991', '1', '1', '1'],
 ['! $17.95', '1992', '5', '5', '5']]

In [45]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("ngram", StringType()),
    StructField("Year", IntegerType()),
    StructField("Count", IntegerType()),
    StructField("Pages", IntegerType()),
    StructField("Books", IntegerType())
])

In [46]:
tableau.take(5)


[['! $17.95', '1985', '1', '1', '1'],
 ['! $17.95', '1987', '1', '1', '1'],
 ['! $17.95', '1990', '1', '1', '1'],
 ['! $17.95', '1991', '1', '1', '1'],
 ['! $17.95', '1992', '5', '5', '5']]

In [47]:
tableau_int = tableau.map(lambda ligne: [ligne[0], int(ligne[1]), int(ligne[2]), int(ligne[3]), int(ligne[4])])



In [48]:
tableau_int.take(5)


[['! $17.95', 1985, 1, 1, 1],
 ['! $17.95', 1987, 1, 1, 1],
 ['! $17.95', 1990, 1, 1, 1],
 ['! $17.95', 1991, 1, 1, 1],
 ['! $17.95', 1992, 5, 5, 5]]

In [49]:
ngram_df = sqlContext.createDataFrame(tableau_int, schema)


In [50]:
ngram_df.show(5)

+--------+----+-----+-----+-----+
|   ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1985|    1|    1|    1|
|! $17.95|1987|    1|    1|    1|
|! $17.95|1990|    1|    1|    1|
|! $17.95|1991|    1|    1|    1|
|! $17.95|1992|    5|    5|    5|
+--------+----+-----+-----+-----+
only showing top 5 rows



In [51]:
ngram_df.printSchema()  # vérifier le schéma


root
 |-- ngram: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Count: integer (nullable = true)
 |-- Pages: integer (nullable = true)
 |-- Books: integer (nullable = true)



In [52]:
ngram_df.registerTempTable("ngram_df")


/usr/local/lib/python3.12/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


**1.Retourner tous les bi-grammes dont le nombre Count est supérieur à cinq**

In [56]:
result1_sql = sqlContext.sql("SELECT * FROM ngram_df WHERE Count > 5")


In [57]:
result1_sql.show()


+--------+----+-----+-----+-----+
|   ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|    ! 09|1899|    6|    6|    5|
|    ! 09|1916|    7|    7|    4|
|    ! 09|1936|    6|    6|    6|
|    ! 09|1997|    6|    5|    5|
|    ! 09|1999|   11|   10|   10|
|    ! 09|2000|   11|    9|    9|
|    ! 09|2004|   14|   14|   14|
|    ! 09|2005|   13|   13|   13|
+--------+----+-----+-----+-----+



R1 avec API SparkSQL

In [60]:
result1_api=ngram_df.filter(ngram_df.Count>5)

In [61]:
result1_api.show()


+--------+----+-----+-----+-----+
|   ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|    ! 09|1899|    6|    6|    5|
|    ! 09|1916|    7|    7|    4|
|    ! 09|1936|    6|    6|    6|
|    ! 09|1997|    6|    5|    5|
|    ! 09|1999|   11|   10|   10|
|    ! 09|2000|   11|    9|    9|
|    ! 09|2004|   14|   14|   14|
|    ! 09|2005|   13|   13|   13|
+--------+----+-----+-----+-----+



** 2) Retourner le nombre total de bi-grammes dans chaque année.\**

In [79]:
result2_sql= sqlContext.sql("SELECT Year,COUNT(Count) AS totalBigrammes from ngram_df GROUP BY Year")

In [80]:
result2_sql.show()

+----+--------------+
|Year|totalBigrammes|
+----+--------------+
|1829|             1|
|1990|             2|
|1884|             1|
|2003|             2|
|1823|             1|
|2007|             2|
|1869|             1|
|1866|             1|
|1877|             1|
|2006|             2|
|1824|             1|
|1848|             1|
|1878|             1|
|1834|             1|
|1997|             2|
|1847|             1|
|1806|             1|
|1856|             1|
|1841|             1|
|1861|             1|
+----+--------------+
only showing top 20 rows



In [75]:
result2_api=ngram_df.groupBy("Year").count()

In [76]:
result2_api.show()

+----+-----+
|Year|count|
+----+-----+
|1829|    1|
|1990|    2|
|1884|    1|
|2003|    2|
|1823|    1|
|2007|    2|
|1869|    1|
|1866|    1|
|1877|    1|
|2006|    2|
|1824|    1|
|1848|    1|
|1878|    1|
|1834|    1|
|1997|    2|
|1847|    1|
|1806|    1|
|1856|    1|
|1841|    1|
|1861|    1|
+----+-----+
only showing top 20 rows



 3) Retourner les bi-grammes qui ont le plus grand nombre de count dans chaque
année



In [98]:
result_sql3 = sqlContext.sql("""
    SELECT ngram, Year, Count
    FROM ngram_df t1
    WHERE Count = (
        SELECT MAX(Count)
        FROM ngram_df t2
        WHERE t1.Year = t2.Year
    )
    ORDER BY Year
""")

result_sql3.show()


+-----+----+-----+
|ngram|Year|Count|
+-----+----+-----+
| ! 09|1780|    1|
| ! 09|1803|    1|
| ! 09|1806|    1|
| ! 09|1823|    1|
| ! 09|1824|    1|
| ! 09|1825|    1|
| ! 09|1829|    3|
| ! 09|1830|    2|
| ! 09|1831|    1|
| ! 09|1833|    1|
| ! 09|1834|    4|
| ! 09|1836|    1|
| ! 09|1839|    1|
| ! 09|1840|    1|
| ! 09|1841|    2|
| ! 09|1845|    1|
| ! 09|1847|    2|
| ! 09|1848|    1|
| ! 09|1856|    1|
| ! 09|1857|    2|
+-----+----+-----+
only showing top 20 rows



In [96]:
from pyspark.sql.functions import col, max as spark_max

# Alias pour les DataFrames
df1 = ngram_df.alias("df1")
df2 = ngram_df.groupBy("Year").agg(spark_max("Count").alias("max_count")).alias("df2")

# Faire le join en qualifiant les colonnes
result_api3 = df1.join(df2, (df1.Year == df2.Year) & (df1.Count == df2.max_count)) \
               .select(df1.ngram, df1.Year, df1.Count) \
               .orderBy(df1.Year)

result_api3.show()

+-----+----+-----+
|ngram|Year|Count|
+-----+----+-----+
| ! 09|1780|    1|
| ! 09|1803|    1|
| ! 09|1806|    1|
| ! 09|1823|    1|
| ! 09|1824|    1|
| ! 09|1825|    1|
| ! 09|1829|    3|
| ! 09|1830|    2|
| ! 09|1831|    1|
| ! 09|1833|    1|
| ! 09|1834|    4|
| ! 09|1836|    1|
| ! 09|1839|    1|
| ! 09|1840|    1|
| ! 09|1841|    2|
| ! 09|1845|    1|
| ! 09|1847|    2|
| ! 09|1848|    1|
| ! 09|1856|    1|
| ! 09|1857|    2|
+-----+----+-----+
only showing top 20 rows



 4) Retourner tous les bi-grammes qui sont apparus dans 20 années différent



In [99]:
from pyspark.sql.functions import countDistinct, col

# Grouper par bi-gramme et compter le nombre d'années distinctes
result_api4 = ngram_df.groupBy("ngram") \
                    .agg(countDistinct("Year").alias("years_count")) \
                    .filter(col("years_count") == 20) \
                    .orderBy(col("years_count").desc())

# Afficher le résultat
result_api4.show(10)

+--------+-----------+
|   ngram|years_count|
+--------+-----------+
|! $17.95|         20|
+--------+-----------+



In [106]:
result_sql4=sqlContext.sql("""
SELECT ngram ,COUNT(DISTINCT Year)
FROM ngram_df
GROUP BY ngram
HAVING COUNT(DISTINCT Year) =20

""")
result_sql4.show()

+--------+--------------------+
|   ngram|count(DISTINCT Year)|
+--------+--------------------+
|! $17.95|                  20|
+--------+--------------------+



5) Retourner tous les bi-grammes qui contiennent le caractère ‘!’ dans la première
 partie et le caractère ‘9’ dans la deuxième partie (les deux parties sont séparées par
un espace).

In [107]:
result_sql5=sqlContext.sql("""SELECT ngram, Year, Count
FROM ngram_df
WHERE SPLIT(ngram, ' ')[0] LIKE '%!%'
  AND SPLIT(ngram, ' ')[1] LIKE '%9%'
  """)
result_sql5.show()

+--------+----+-----+
|   ngram|Year|Count|
+--------+----+-----+
|! $17.95|1985|    1|
|! $17.95|1987|    1|
|! $17.95|1990|    1|
|! $17.95|1991|    1|
|! $17.95|1992|    5|
|! $17.95|1993|    2|
|! $17.95|1995|    1|
|! $17.95|1996|    4|
|! $17.95|1997|    6|
|! $17.95|1998|    4|
|! $17.95|1999|   11|
|! $17.95|2000|   11|
|! $17.95|2001|    5|
|! $17.95|2002|    5|
|! $17.95|2003|    2|
|! $17.95|2004|   14|
|! $17.95|2005|   13|
|! $17.95|2006|    5|
|! $17.95|2007|    2|
|! $17.95|2008|    2|
+--------+----+-----+
only showing top 20 rows



In [109]:
from pyspark.sql.functions import split, col

# Séparer le bi-gramme en deux parties
parts = split(col("ngram"), " ")

# Filtrer selon les conditions
result_api5 = ngram_df.withColumn("first_part", parts.getItem(0)) \
                    .withColumn("second_part", parts.getItem(1)) \
                    .filter(col("first_part").contains("!")) \
                    .filter(col("second_part").contains("9")) \
                    .select("ngram", "Year", "Count")  # afficher uniquement les colonnes utiles

result_api5.show()

+--------+----+-----+
|   ngram|Year|Count|
+--------+----+-----+
|! $17.95|1985|    1|
|! $17.95|1987|    1|
|! $17.95|1990|    1|
|! $17.95|1991|    1|
|! $17.95|1992|    5|
|! $17.95|1993|    2|
|! $17.95|1995|    1|
|! $17.95|1996|    4|
|! $17.95|1997|    6|
|! $17.95|1998|    4|
|! $17.95|1999|   11|
|! $17.95|2000|   11|
|! $17.95|2001|    5|
|! $17.95|2002|    5|
|! $17.95|2003|    2|
|! $17.95|2004|   14|
|! $17.95|2005|   13|
|! $17.95|2006|    5|
|! $17.95|2007|    2|
|! $17.95|2008|    2|
+--------+----+-----+
only showing top 20 rows



6) Retourner les bi-grammes qui sont apparus dans toutes les années présentes
dans les données.

In [118]:
result_sql6=sqlContext.sql("""SELECT ngram
FROM ngram_df
GROUP BY ngram
HAVING COUNT(DISTINCT Year) = (
    SELECT COUNT(DISTINCT Year) FROM ngram_df
)""")
result_sql6.show()




+-----+
|ngram|
+-----+
| ! 09|
+-----+



In [111]:
from pyspark.sql.functions import countDistinct, col

# Étape 1 : trouver le nombre total d'années dans le dataset
total_years = ngram_df.select("Year").distinct().count()

# Étape 2 : filtrer les bi-grammes présents dans toutes ces années
result_api6 = (
    ngram_df.groupBy("ngram")
            .agg(countDistinct("Year").alias("years_count"))
            .filter(col("years_count") == total_years)
)

result_api6.show()

+-----+-----------+
|ngram|years_count|
+-----+-----------+
| ! 09|        100|
+-----+-----------+



Retourner le nombre total de pages et de livres dans lesquels chaque bi
gramme apparaît pour chaque année disponible, trié par ordre alphabétique

In [120]:
result_sql7 = sqlContext.sql("""
    SELECT
        ngram,
        Year,
        COUNT(DISTINCT Pages) AS total_pages,
        COUNT(DISTINCT Books) AS total_books
    FROM ngram_df
    GROUP BY ngram, Year
    ORDER BY ngram ASC
""")
result_sql7.show()


+--------+----+-----------+-----------+
|   ngram|Year|total_pages|total_books|
+--------+----+-----------+-----------+
|! $17.95|2002|          1|          1|
|! $17.95|2007|          1|          1|
|! $17.95|1985|          1|          1|
|! $17.95|1990|          1|          1|
|! $17.95|1993|          1|          1|
|! $17.95|2003|          1|          1|
|! $17.95|2000|          1|          1|
|! $17.95|2005|          1|          1|
|! $17.95|1999|          1|          1|
|! $17.95|1992|          1|          1|
|! $17.95|1996|          1|          1|
|! $17.95|1997|          1|          1|
|! $17.95|1995|          1|          1|
|! $17.95|2004|          1|          1|
|! $17.95|1987|          1|          1|
|! $17.95|2006|          1|          1|
|! $17.95|1998|          1|          1|
|! $17.95|1991|          1|          1|
|! $17.95|2008|          1|          1|
|! $17.95|2001|          1|          1|
+--------+----+-----------+-----------+
only showing top 20 rows



In [121]:
result_sql8 = sqlContext.sql("""
    SELECT
        Year,
        COUNT(DISTINCT ngram) AS total_bigrams
    FROM ngram_df
    GROUP BY Year
    ORDER BY Year DESC
""")
result_sql8.show()

+----+-------------+
|Year|total_bigrams|
+----+-------------+
|2008|            2|
|2007|            2|
|2006|            2|
|2005|            2|
|2004|            2|
|2003|            2|
|2002|            2|
|2001|            2|
|2000|            2|
|1999|            2|
|1998|            2|
|1997|            2|
|1996|            2|
|1995|            2|
|1993|            2|
|1992|            2|
|1991|            2|
|1990|            2|
|1987|            2|
|1985|            2|
+----+-------------+
only showing top 20 rows



In [122]:
from pyspark.sql.functions import countDistinct

result_api8 = (
    ngram_df.groupBy("Year")
            .agg(countDistinct("ngram").alias("total_bigrams"))
            .orderBy("Year", ascending=False)
)

result_api8.show()

+----+-------------+
|Year|total_bigrams|
+----+-------------+
|2008|            2|
|2007|            2|
|2006|            2|
|2005|            2|
|2004|            2|
|2003|            2|
|2002|            2|
|2001|            2|
|2000|            2|
|1999|            2|
|1998|            2|
|1997|            2|
|1996|            2|
|1995|            2|
|1993|            2|
|1992|            2|
|1991|            2|
|1990|            2|
|1987|            2|
|1985|            2|
+----+-------------+
only showing top 20 rows

